# eFarmersHub Data Analysis
eFarmersHub data are stored in `gds_database` for data visualization. There are 8 tables:
1. Income generating tables: Sale, Machine Rent & Advisory
2. Expenditure tables: Purchase, Processing, Expense
3. User table: This table stores all the user data

The script utilizes `SQLAlchemy` as a database toolkit for CRUD operation while `Pandas` is used for data manipulation.

In [153]:
# Import Modules
# data manipulation and analysis
import pandas as pd
import numpy as np

# database toolkit
from sqlalchemy import create_engine, MetaData, inspect, Table, Column, Integer, String, Date, Numeric, extract
from sqlalchemy.engine.url import URL
from sqlalchemy.sql import select

# read env file
from dotenv import load_dotenv
import os

# path handling
from pathlib import Path

# logging
import logging

# z-score for anomaly detection
from scipy.stats import zscore

In [154]:
# load env variables
dotenv_path = Path("./.env")
load_dotenv(dotenv_path=dotenv_path)

USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")

### 1. Read Sale Table
Sale data are stored in `gds_sale_transactions` table. For financial analysis such as revenue and profit, `net_amount` and `cogs_amount` are considered.

In [155]:
def extract_sale(engine):
    """
    read gds_sale_transactions table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, market_type,
                    business_category, category, product, transaction_date, transaction_id, customer_id, customer_name,
                    customer_mobile, customer_gender, product_amount, net_amount, due_amount, cogs_amount, version,
                    currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, customer_id, customer_name, customer_mobile, customer_gender,
                        market_type, sale_type, business_category, category_id, category, product_id, product,
                        unit_type, attribute, quantity, unit_price, product_amount, sub_total_amount,
                        commission_amount, discount_amount, net_amount, paid_amount, due_amount, due_receivable_date,
                        version, user_join_date, user_region, customer_join_date, currency_exchange_rate, cogs_amount
                    FROM gds_database.gds_sale_transactions
                ) sale;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s",
            level=logging.ERROR)
        logging.error(e)
        
    return df

In [156]:
def transform_sale(df):
    """
    transform sale dataframe and returns df
    :param df: actual sale dataframe
    :return df: transformed dataframe
    """
    
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)
    
    # convert date_of_transaction to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["customer_id"] = df["customer_id"].astype(str)
    df["customer_mobile"] = df["customer_mobile"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["product_amount"] = df["product_amount"].astype(float)
    df["net_amount"] = df["net_amount"].astype(float)
    df["cogs_amount"] = df["cogs_amount"].astype(float)
    df["due_amount"] = df["due_amount"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)

    # renaming columns for consistency
    df.rename(columns={"net_amount" : "revenue"}, inplace=True)

    # add transaction_category column to identify the module used for transaction
    df["transaction_category"] = "Sale"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category", "product",
            "version"]) \
        .drop_duplicates(subset=["transaction_id", "category", "product"], keep="last")

    return df

### 2. Machine Rent
Machine Rent data are stored in `gds_machine_rent_transactions` table. For financial analysis such as revenue and profit, `net_amount` is considered.

**Note:** Depreciation is not being considered at the moment.

In [157]:
def extract_machine_rent(engine):
    """
    read gds_machine_rent_transactions table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, business_category,
                    category, product, transaction_date, transaction_id, customer_id, customer_name,
                    customer_mobile, customer_gender, amount, net_amount, due_amount, version, currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, customer_id, customer_name, customer_mobile, customer_gender,
                        business_category, category_id, category, product_id, product, unit_type, quantity,
                        unit_price, unit_count, amount, sub_total_amount, net_amount, paid_amount, due_amount,
                        due_receivable_date, land_type, land_size, start_date_time, end_date_time, rent_hour,
                        version, user_join_date, user_region, customer_join_date, currency_exchange_rate
                    FROM gds_database.gds_machine_rent_transactions
                ) machine_rent;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s",
            level=logging.ERROR)
        logging.error(e)
        
    return df

In [158]:
def transform_machine_rent(df):
    """
    transform machine_rent dataframe and returns df
    :param df: actual machine_rent dataframe
    :return df: transformed dataframe
    """
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)

    # convert date_of_transaction to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["customer_id"] = df["customer_id"].astype(str)
    df["customer_mobile"] = df["customer_mobile"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["amount"] = df["amount"].astype(float)
    df["net_amount"] = df["net_amount"].astype(float)
    df["due_amount"] = df["due_amount"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)
    
    # renaming columns for consistency
    df.rename(columns={"net_amount" : "revenue",
        "amount": "product_amount"}, inplace=True)

    # add market_type column
    df["market_type"] = "Farmer"
    df["transaction_category"] = "Machinery Rental"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category", "product",
            "version"]) \
        .drop_duplicates(subset=["transaction_id", "category", "product"], keep="last")

    return df

### 3. Advisory Service
Advisory data are stored in `gds_advisory_transactions` table. For financial analysis such as revenue and profit, `amount` is considered.

In [159]:
def extract_advisory(engine):
    """
    read advisory table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, business_categories,
                    categories, transaction_date, transaction_id, customer_id, customer_name, customer_mobile,
                        customer_gender, amount, version, currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, user_join_date, user_region,
                        parent_name, transaction_id, transaction_date, customer_id, customer_name, customer_mobile,
                        customer_gender, customer_join_date, categories_ids, business_categories, categories,
                        tags_ids, tags, comments, amount, usd_amount, version, currency_exchange_rate
                    FROM gds_database.gds_advisory_transactions
                ) advisory;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s",
            level=logging.ERROR)
        logging.error(e)
        
    return df

In [160]:
def transform_advisory(df):
    """
    transform advisory dataframe and returns df
    :param df: actual machine_rent dataframe
    :return df: transformed dataframe
    """
    
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)

    # convert date_of_transaction to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["customer_id"] = df["customer_id"].astype(str)
    df["customer_mobile"] = df["customer_mobile"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["amount"] = df["amount"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)
    
    # renaming columns for consistency
    df.rename(columns={"amount" : "revenue"}, inplace=True)

    # add market_type column
    df["market_type"] = "Farmer"
    df["transaction_category"] = "Advisory"
    df["business_category"] = "Advisory"
    df["category"] = "Advisory"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category",
        "version"]) \
        .drop_duplicates(subset=["transaction_id", "category"], keep="last")

    return df

### 4. Purchase
Purchase data are stored in `gds_purchase_transactions` table.

In [161]:
def extract_purchase(engine):
    """
    read purchase table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, market_type,
                    business_category, category, product, transaction_date, transaction_id, supplier_id, supplier_name,
                    supplier_mobile, supplier_gender, product_amount, net_amount, due_amount, version,
                    currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, supplier_id, supplier_name, supplier_mobile, supplier_gender, market_type,
                        business_category, category_id, category, product_id, product, unit_type, attribute, quantity,
                        unit_price, product_amount, sub_total_amount, commission_amount, net_amount, paid_amount,
                        due_amount, due_payable_date, version, user_join_date, user_region, supplier_join_date,
                        currency_exchange_rate
                    FROM gds_database.gds_purchase_transactions
                ) purchase;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s", level=logging.ERROR)
        logging.error(e)
        
    return df

In [162]:
def transform_purchase(df):
    """
    transform purchase dataframe and returns df
    :param df: actual purchase dataframe
    :return df: transformed dataframe
    """
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)
    
    # convert transaction_date to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["supplier_id"] = df["supplier_id"].astype(str)
    df["supplier_mobile"] = df["supplier_mobile"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["net_amount"] = df["net_amount"].astype(float)
    df["due_amount"] = df["due_amount"].astype(float)   
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)

    # renaming columns for consistency
    df.rename(columns={"net_amount" : "expenditure",
        "product_amount" : "product_expenditure"}, inplace=True)

    # add market_type column
    df["transaction_category"] = "Purchase"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category", "product",
            "version"]) \
        .drop_duplicates(subset=["transaction_id", "category", "product"], keep="last")

    return df

### 5. Processing
Processing data are stored in `gds_processing_transactions` table.

In [163]:
def extract_processing(engine):
    """
    read processing table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, business_category,
                    category, product, transaction_date, transaction_id, amount, production_cost, version,
                    currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, business_category, category_id, category, product_id, product, unit_type,
                        quantity, unit_price, amount, production_cost, version, user_join_date, user_region,
                        currency_exchange_rate
                    FROM gds_database.gds_processing_transactions
                ) processing;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s", level=logging.ERROR)
        logging.error(e)
        
    return df

In [164]:
def transform_processing(df):
    """
    transform processing dataframe and returns df
    :param df: actual processing dataframe
    :return df: transformed dataframe
    """
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)
    
    # convert transaction_date to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["amount"] = df["amount"].astype(float)
    df["production_cost"] = df["production_cost"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)

    # renaming columns for consistency
    df.rename(columns={"production_cost" : "expenditure",
        "amount" : "product_expenditure"}, inplace=True)

    # add transaction category column
    df["market_type"] = df["user_type"]
    df["transaction_category"] = "Processing"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category", "product",
            "version"]) \
        .drop_duplicates(subset=["transaction_id", "category", "product"], keep="last")

    return df

### 6. Machine Purchase
Machine Purchase data are stored in `gds_machine_purchase_transactions` table.

In [165]:
def extract_machine_purchase(engine):
    """
    read machine_purchase table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, business_category,
                    category, product, transaction_date, transaction_id, supplier_id, supplier_name,
                    supplier_mobile, supplier_gender, total_amount, due_amount, version,
                    currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, supplier_id, supplier_name, supplier_mobile, supplier_gender,
                        business_category, category_id, category, product_id, product, quantity, unit_price,
                        total_amount, paid_amount, due_amount, due_payable_date, version, user_join_date, user_region,
                        supplier_join_date, currency_exchange_rate
                    FROM gds_database.gds_machine_purchase_transactions
                ) machine_purchase;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s",
            level=logging.ERROR)
        logging.error(e)
        
    return df

In [166]:
def transform_machine_purchase(df):
    """
    transform machine_purchase dataframe and returns df
    :param df: actual machine_purchase dataframe
    :return df: transformed dataframe
    """
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)
    
    # convert transaction_date to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["supplier_id"] = df["supplier_id"].astype(str)
    df["supplier_mobile"] = df["supplier_mobile"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["total_amount"] = df["total_amount"].astype(float)
    df["due_amount"] = df["due_amount"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)    

    # usd conversion
    df.rename(columns={"total_amount" : "expenditure"}, inplace=True)

    # add transaction category column
    df["market_type"] = df["user_type"]
    df["transaction_category"] = "Machinery Purchase"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "category", "product",
            "version"]) \
        .drop_duplicates(subset=["transaction_id"], keep="last")

    return df

### 7. Expense
Expense data are stored in `gds_expense_transactions` table.

In [167]:
def extract_expense(engine):
    """
    read expense table from sql database and returns df
    :param engine: SQLAlchemy engine object
    :return df: sale dataframe
    """
    try:
        with engine.connect() as conn:
            query = """
                SELECT country_name, parent_name, user_type, user_region, user_name, user_id, expense_category,
                    business_category, product_category, expense_type, transaction_date, transaction_id, total_amount,
                    version, currency_exchange_rate
                FROM (
                    SELECT distinct user_id, country_name, user_name, user_type, parent_name, transaction_id,
                        transaction_date, expense_category, expense_type, total_amount, business_category,
                        product_category_id, product_category, version, user_join_date, user_region,
                        currency_exchange_rate
                    FROM gds_database.gds_expense_transactions
                ) expense;
                """
            df = pd.read_sql(query, conn)
    except Exception as e:
        logging.basicConfig(filename="./log", filemode="a", format="%(asctime)s - %(levelname)s - %(message)s",
            level=logging.ERROR)
        logging.error(e)
        
    return df

In [168]:
def transform_expense(df):
    """
    transform expense dataframe and returns df
    :param df: actual expense dataframe
    :return df: transformed dataframe
    """
    # drop duplicates
    df.drop_duplicates(inplace=True, ignore_index=True)
    
    # convert transaction_date to datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], format="%Y/%m/%d")

    # convert user_id to string
    df["user_id"] = df["user_id"].astype(str)
    df["transaction_id"] = df["transaction_id"].astype(str)

    # convert and round numerical columns
    df["total_amount"] = df["total_amount"].astype(float)
    df["currency_exchange_rate"] = df["currency_exchange_rate"].astype(float)

    # revenue & profit
    df.rename(columns={"total_amount" : "expenditure",
        "product_category" : "category"}, inplace=True)

    # add transaction category column
    df["market_type"] = df["user_type"]
    df["transaction_category"] = "Expense"

    # sorting data based on version and keep the latest version only
    df = df.sort_values(["country_name", "parent_name", "user_id", "transaction_id", "expense_category",
            "category", "expense_type", "version"]) \
        .drop_duplicates(subset=["transaction_id", "expense_category", "category", "expense_type"],
            keep="last")

    return df

In [169]:
def anomaly_income(df):
    """
    identify data anomaly especially net_amount
    :param df: transaction dataframe
    :return df: filtered df
    """
    agg_df = df.groupby(["transaction_category", "user_name", "user_id", "transaction_id"]) \
        .agg(total_revenue=("revenue", "sum")).reset_index()
    agg_df["z_score"] = agg_df.groupby(["user_id", "transaction_category"])["total_revenue"] \
        .transform(lambda x : zscore(x, ddof=1))
    anomaly = agg_df.loc[abs(agg_df["z_score"]) > 3]
    transaction_id_list = anomaly["transaction_id"].tolist()
    mask = df["transaction_id"].isin(transaction_id_list)
    df = df[~mask]

    return df

In [170]:
def anomaly_expenditure(df):
    """
    identify data anomaly especially net_amount
    :param df: transaction dataframe
    :return df: filtered df
    """
    agg_df = df.groupby(["transaction_category", "user_name", "user_id", "transaction_id"]) \
        .agg(total_expenditure=("expenditure", "sum")).reset_index()
    agg_df["z_score"] = agg_df.groupby(["user_id", "transaction_category"])["total_expenditure"] \
        .transform(lambda x : zscore(x, ddof=1))
    anomaly = agg_df.loc[abs(agg_df["z_score"]) > 3]
    transaction_id_list = anomaly["transaction_id"].tolist()
    mask = df["transaction_id"].isin(transaction_id_list)
    df = df[~mask]

    return df

In [171]:
if __name__ == "__main__":
    # initiate connection to database
    connect_url = URL.create(
        "mysql+pymysql",
        username=USERNAME,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        database=DATABASE
    )
    engine = create_engine(connect_url)

    # debug
    # with engine.connect() as conn:
    #     inspector = inspect(engine)
    #     table_names = inspector.get_table_names()
    #     print(table_names)

    # sale
    sale = extract_sale(engine)
    sale = transform_sale(sale)
    # sale.to_csv("../output/sale.csv", index=False)

    # machine rent
    machine_rent = extract_machine_rent(engine)
    machine_rent = transform_machine_rent(machine_rent)
    # machine_rent.to_csv("../output/machine_rent.csv", index=False)

    # advisory
    advisory = extract_advisory(engine)
    advisory = transform_advisory(advisory)
    # advisory.to_csv("../output/advisory.csv", index=False)

    # purchase
    purchase = extract_purchase(engine)
    purchase = transform_purchase(purchase)
    # purchase.to_csv("../output/purchase.csv", index=False)

    # processing
    processing = extract_processing(engine)
    processing = transform_processing(processing)
    # processing.to_csv("../output/processing.csv", index=False)

    # machine purchase
    machine_purchase = extract_machine_purchase(engine)
    machine_purchase = transform_machine_purchase(machine_purchase)
    # machine_purchase.to_csv("../output/machine_purchase.csv", index=False)

    # expense
    expense = extract_expense(engine)
    expense = transform_expense(expense)
    # expense.to_csv("../output/expense.csv", index=False)

    # concat all cash-in transactions
    income = pd.concat([sale, machine_rent, advisory], sort=False, ignore_index=True)
    income.to_csv("../output/income_actual.csv", index=False)
    income = anomaly_income(income)
    # usd conversion
    income["product_amount_usd"] = round(income["product_amount"] / income["currency_exchange_rate"], 4)
    income["revenue_usd"] = round(income["revenue"] / income["currency_exchange_rate"], 4)
    income["cogs_amount_usd"] = round(income["cogs_amount"] / income["currency_exchange_rate"], 4)
    # data cleaning for outliers
    income = income.loc[(income["revenue_usd"] < 10000) & (income["cogs_amount_usd"] < 10000)]
    income.to_csv("../output/income_filter.csv", index=False)

    # concat all cash-out transactions
    expenditure = pd.concat([purchase, machine_purchase, processing], sort=False, ignore_index=True)
    expenditure.to_csv("../output/expenditure_actual.csv", index=False)
    expenditure = anomaly_expenditure(expenditure)
    # usd conversion
    expenditure["product_expenditure_usd"] = round(expenditure["product_expenditure"] / 
        expenditure["currency_exchange_rate"], 4)
    expenditure["expenditure_usd"] = round(expenditure["expenditure"] / expenditure["currency_exchange_rate"], 4)
    expenditure.to_csv("../output/expenditure_filter.csv", index=False)
    # data cleaning for outliers
    expenditure = expenditure.loc[expenditure["expenditure_usd"] < 10000]
    expenditure.to_csv("../output/expenditure_filter.csv", index=False)

    # filter direct cost
    direct_cost = expense.loc[expense["expense_category"] == "Direct Cost"]
    direct_cost.to_csv("../output/direct_cost_actual.csv", index=False)
    direct_cost = anomaly_expenditure(direct_cost)
    # usd conversion
    direct_cost["direct_cost_usd"] = round(direct_cost["expenditure"] / direct_cost["currency_exchange_rate"], 4)
    # data cleaning for outliers
    direct_cost = direct_cost.loc[direct_cost["direct_cost_usd"] < 10000]
    direct_cost.to_csv("../output/direct_cost_filter.csv", index=False)

    # filter indirect cost
    indirect_cost = expense.loc[expense["expense_category"] == "Indirect Cost"]
    indirect_cost.to_csv("../output/indirect_cost_actual.csv", index=False)
    indirect_cost = anomaly_expenditure(indirect_cost)
    # usd conversion
    indirect_cost["indirect_cost_usd"] = round(indirect_cost["expenditure"] / indirect_cost["currency_exchange_rate"], 4)
    # data cleaning for outliers
    indirect_cost = indirect_cost.loc[indirect_cost["indirect_cost_usd"] < 10000]
    indirect_cost.to_csv("../output/indirect_cost_filter.csv", index=False)
    expense = pd.concat([direct_cost, indirect_cost], sort=False, ignore_index=True)
    expense.drop(columns=["expense_type"], inplace=True)

    master_data = pd.concat([income, expenditure, expense], sort=False, ignore_index=True)
    master_data.drop(columns=["business_categories", "categories"], inplace=True)
    master_data.to_csv("../output/master_data.csv", index=False)
    master_data.to_sql("master_data_global", con=engine, if_exists='replace', index = False)

    # # cumulative revenue & profit
    # cash_flow = income.groupby(["country_name", "user_type", "parent_name", "user_region", "user_name",
    #     "transaction_date"]) \
    #     .agg(unique_income_transaction=("transaction_id", "nunique"),
    #         revenue=("revenue", "sum"),
    #         currency_exchange_rate=("currency_exchange_rate", "mean"),
    #         total_cogs=("cogs_amount", "sum")).reset_index()
    
    # # filter direct cost
    # direct_cost = expense.loc[expense["expense_category"] == "Direct Cost"]
    # direct_cost = direct_cost.groupby(["country_name", "user_type", "parent_name", "user_region", "user_name",
    #     "transaction_date"]) \
    #     .agg(total_direct_cost=("expenditure", "sum")).reset_index()

    # # filter indirect cost
    # indirect_cost = expense.loc[expense["expense_category"] == "Indirect Cost"]
    # indirect_cost = indirect_cost.groupby(["country_name", "user_type", "parent_name", "user_region", "user_name",
    #     "transaction_date"]) \
    #     .agg(total_indirect_cost=("expenditure", "sum")).reset_index()

    # # expenditure
    # expenditure = pd.concat([purchase, machine_purchase, processing], sort=False, ignore_index=True)

    # # cumulative expenditure
    # expenditure = expenditure.groupby(["country_name", "user_type", "parent_name", "user_region", "user_name",
    #     "transaction_date"]) \
    #     .agg(unique_expenditure_transaction=("transaction_id", "nunique"),
    #         expenditure=("expenditure", "sum")).reset_index()

    # # join expenditure with revenue table
    # cash_flow = cash_flow.merge(expenditure, how="outer", on=["country_name", "user_type", "parent_name",
    #     "user_region", "user_name", "transaction_date"])

    # # join expenditure with revenue table
    # cash_flow = cash_flow.merge(direct_cost, how="outer", on=["country_name", "user_type",
    #     "parent_name", "user_region", "user_name", "transaction_date"])

    # cash_flow = cash_flow.merge(indirect_cost, how="outer", on=["country_name", "user_type",
    #     "parent_name", "user_region", "user_name", "transaction_date"])

    # cash_flow = cash_flow.sort_values(["country_name", "parent_name", "user_name",
    #     "transaction_date"])
    
    # # usd conversion
    # cash_flow["revenue_usd"] = round(cash_flow["revenue"] / cash_flow["currency_exchange_rate"] , 4)
    # cash_flow["cogs_usd"] = round(cash_flow["total_cogs"] / cash_flow["currency_exchange_rate"] , 4)
    # cash_flow["total_direct_cost_usd"] = round(cash_flow["total_direct_cost"] / cash_flow["currency_exchange_rate"] , 4)
    # cash_flow["total_indirect_cost_usd"] = round(cash_flow["total_indirect_cost"] / cash_flow["currency_exchange_rate"] , 4)
    # cash_flow["expenditure_usd"] = round(cash_flow["expenditure"] / cash_flow["currency_exchange_rate"] , 4)

    # # filter transactions that are more than 10000 USD
    # cash_flow = cash_flow.loc[cash_flow["revenue_usd"] < 10000]
    # cash_flow = cash_flow.loc[cash_flow["total_direct_cost_usd"] < 10000]
    # cash_flow = cash_flow.loc[cash_flow["total_indirect_cost_usd"] < 10000]
    # cash_flow = cash_flow.loc[cash_flow["expenditure_usd"] < 10000]

    # cash_flow.to_sql("cash_flow_global", con=engine, if_exists='replace', index = False)
    # # cash_flow.to_sql("cash_flow_global", con=engine, if_exists='replace', index = False)
    # cash_flow.to_csv("../output/cash_flow.csv", index=False)